#My_Lane_Finding_Advanced-Notebook
Advanced Lane Detection Term 1

Self Driving Car NanoDegree

Please note that the Pipeline has been modified for the various functions to work independently.
For a Complete Pipeline, Please check the main.py file along with

1. class LineDetector(object),

2. class PerspectiveTransform(),

3. class Line(object).

In [0]:
"""
Advanced Lane Detection Term 1
Self Driving Car NanoDegree

"""

In [0]:
#Import Dependencies
import sys
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [0]:
#Camera Calibration
# This function computes camera Calibration Parameters
# 1. Calibration Matrix
# 2. Distortion Coefficients
def GetCalibrationParam(image_url):

    images = glob.glob(image_url)   #store images

    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    object_points = [] # 3d Points in real world space
    image_points = [] # 2d Points in image plane.
    corner = (9, 6) # Chessboard size to 9x6

    # Iterate over the stored images
    for image in images:
        img = mpimg.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, corner, None)

        if ret:
            object_points.append(objp)
            image_points.append(corners)

    img_size = (img.shape[1], img.shape[0])

    # Here, we will use built in cv2 function named as calibrateCamera
    # This function finds the camera intrinsic and extrinsic parameters...
    # from several views of a calibration pattern
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, img_size,None,None)

    # Return Calibration matrix and Distortion Matrix
    return mtx, dist

# This function returns undistorted images using the calibration and distortion matrices
def GetUndistortion(distorted_img, mtx, dist):
# Here, we will use the built in cv2 function named as undistort
# This function transforms an image to compensate for lens distortion.
    undist = cv2.undistort(distorted_img, mtx, dist, None, mtx)
    return undist   #return the compenstaed image

# This function gets camera callibration matrix and distortion coefficents from the saved configuration file.
def InputCalibrationFile(path="./cal.npz"):
    try:
        calibration_param = np.load(path)
        return calibration_param['mtx'], calibration_param['dist']
    except IOError as e:
        print(e)    # Throw exception
        raise IOError("Please Set Correct Calibration File")

In [11]:
#Calibrate from the camera_cal folder
#Save the Results as .npz file.

if __name__ == "__main__":
  camera_matrix, dist_coeff = GetCalibrationParam('./camera_cal/calibration*.jpg')
  np.savez("./cal.npz",mtx=camera_matrix, dist=dist_coeff)

UnboundLocalError: ignored

In [0]:
#Distorted Image Correction
try:
    calibration_param = np.load('./cal.npz')
except IOError as e:
    print("There is no file in given Path")
else:
    mtx = calibration_param['mtx']
    dist = calibration_param['dist']
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 5))
    for i in range(2):
        for j in range(3):
            image = mpimg.imread("./test_images/test{}.jpg".format(str(i*3 + j + 1)))
            undist_img = get_undistortion(image, mtx, dist)
            axes[i, j].imshow(undist_img)
    fig.tight_layout()
    [ax.axis('off') for axe in axes for ax in axe]
    fig.subplots_adjust(left = None, right = None, top = None, bottom = None, wspace = 0.1, hspace = 0.1)

In [0]:
# Perspective Transform
class PerspectiveTransform():
    def __init__(self, src, dst):
        self.src = src
        self.dst = dst
        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        self.inverse_M = cv2.getPerspectiveTransform(self.dst, self.src)

    ### Here, we used the cv2 function warpPerspective
    #Applies a perspective transformation to an image.

    # This function returns a transformed image
    def Transform(self, undist):
        return cv2.warpPerspective(undist, self.M, (undist.shape[1], undist.shape[0]))

    # This function performs inverse of 'Transform' function, it returns original image.
    def InverseTransform(self, undist):
        return cv2.warpPerspective(undist, self.inverse_M, (undist.shape[1], undist.shape[0]))

In [0]:
#Visualization
try:
    calibration_param = input_calibration_file()
except IOError as e:
    print(e)
else:
    mtx = calibration_param['mtx']
    dist = calibration_param['dist']
    
    # The source and destination points are basically selected by default(hardcoded) as,
    src = np.float32([[490, 482],[810, 482],
                  [1250, 720],[40, 720]])

    dst = np.float32([[0, 0], [1280, 0], 
                     [1250, 720],[40, 720]])

    BirdViewTransformer = PerspectiveTransform(src, dst)
    
    fig, axes = plt.subplots(2, 3, figsize=(12, 5))
    for i in range(2):
        for j in range(3):
            image = mpimg.imread("./test_images/test{}.jpg".format(str(i*3 + j + 1)))
            undist_img = get_undistortion(image, mtx, dist)
            bird_view = BirdViewTransformer.Transform(undist_img)
            axes[i, j].imshow(bird_view)
    fig.tight_layout()
    [ax.axis('off') for axe in axes for ax in axe]
    fig.subplots_adjust(left = None, right = None, top = None, bottom = None, wspace = 0.1, hspace = 0.1)

In [0]:
#Color Thresholding, Binary Thresholding

# This function applies Gaussian Noise Kernal to the given binary image
def GaussianBlurr(img, kernel_size):
    # We use cv2 function GaussianBlur
    # The Gaussian filter is a low-pass filter that removes the high-frequency components are reduced.
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


# Computes S Binary Image (x, y, 1) from H and S of HLS.
def GetSBinary(undist_img, thres=(110, 255)):
    # Use cv2.cvtColor and RGV2HLS Parameter
    hls = cv2.cvtColor(undist_img, cv2.COLOR_RGB2HLS)
    h = hls[:, :, 0]
    s = hls[:, :, 2]
    s_binary = np.zeros_like(s)
    s_binary[((s >= thres[0]) & (s <= thres[1])) & (h <= 30)] = 1
    s_binary = GaussianBlurr(s_binary, kernel_size=21)

    return s_binary

# This function returns RGB image after applying gamma conversion
def AdjustGamma(image, gamma=1.0):

    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    
    # We wll use a look up table LUT from cv2
    return cv2.LUT(image, table)

# This function Calculates Direct Threshold
def DirectThreshold(img_ch, sobel_kernel=3, thresh=(0, np.pi/2)):
    sobelx = np.absolute(cv2.Sobel(img_ch, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(img_ch, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    abs_grad_dir = np.absolute(np.arctan(sobely/sobelx))
    dir_binary =  np.zeros_like(abs_grad_dir)
    dir_binary[(abs_grad_dir > thresh[0]) & (abs_grad_dir < thresh[1])] = 1

    return dir_binary

# This function is basically for Gradent Thresholding
# We First apply gamma conversion to imags for decrease noise using 'AdjustGamma' function.
def GetSlope(undist_img, orient='x', sobel_kernel=3, thres = (0, 255)):

    undist_img = AdjustGamma(undist_img, 0.2)
    gray = cv2.cvtColor(undist_img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        slope = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    elif orient == 'y':
        slope = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    else:
        raise KeyError("select 'x' or 'y'")

    scale_factor = np.max(slope) / 255
    scale_slope = (slope / scale_factor).astype(np.uint8)
    slope_binary = np.zeros_like(scale_slope)
    slope_binary[(scale_slope >= thres[0]) & (scale_slope <= thres[1])] = 1

    slope_binary = GaussianBlurr(slope_binary, kernel_size=9)
    return slope_binary

# Given an undistorted image, return converted image using Color Slope Conversion
def ColorSlopeThresConversion(undist_img):
    s_binary =GetSBinary(undist_img, thres=(150, 255))
    slope = GetSlope(undist_img, orient='x',sobel_kernel=7, thres=(25, 255))

    color_binary = np.zeros_like(s_binary)
    color_binary[(s_binary == 1) | (slope == 1)] = 1

    return slope, color_binary, s_binary

In [0]:
#Visualization
try:
    calibration_param = InputCalibrationFile()
except IOError as e:
    print(e)
else:
    mtx = calibration_param['mtx']
    dist = calibration_param['dist']
    
    src = np.array([[490, 482],[810, 482],
                  [1250, 720],[40, 720]], dtype=np.float32)
    dst = np.array([[0, 0], [1280, 0], 
                     [1250, 720],[40, 720]], dtype=np.float32)
    
    BirdViewTransformer = PerspectiveTransform(src, dst)

    for path in glob.glob("./test_images/test*.jpg"):
        fig, axes = plt.subplots(2, 3, figsize=(10, 4))
        distorted_img = mpimg.imread(path)
        undist_img = get_undistortion(distorted_img, mtx, dist)
        bird_view = bird_view_transformer.transform(undist_img)            
        s_binary, slope, conversion_img = color_slope_thres_conversion(bird_view)
        
        gamma = adjust_gamma(bird_view, 0.2)
        
        axes[0, 0].imshow(undist_img)
        axes[0, 0].set_title("Undistorted Image")
        axes[0, 1].imshow(bird_view)
        axes[0, 1].set_title("Bird's View")
        axes[0, 2].imshow(s_binary, cmap='gray')
        axes[0, 2].set_title("S Binary")
        axes[1, 0].imshow(slope, cmap='gray')
        axes[1, 0].set_title("Slope Binary")
        axes[1, 1].imshow(conversion_img, cmap='gray')
        axes[1, 1].set_title("Combined Binary")
        axes[1, 2].imshow(gamma, cmap='gray')
        axes[1, 2].set_title("ex:) apply gamma conversion")

        fig.tight_layout()
        [ax.axis('off') for axe in axes for ax in axe]
        fig.subplots_adjust(left = None, right = None, top = 1, bottom = None, wspace = 0.1, hspace = 0.1)

In [0]:
#Histogram Filtering
# We define a subfunction for calculating index of max sum value of each histogram.
    def GetMaxIndexHistogram(self, histogram, left_boundary, right_boundary, window_width=10):
        index_list = []
        side_histogram = histogram[left_boundary : right_boundary]
        for i in range(len(side_histogram) - window_width):
            index_list.append(np.sum(side_histogram[i : i + window_width]))
        index = np.argmax(index_list) + int(window_width / 2) + left_boundary
        return index

    # This function calculates Histogram Thresholding for decreasing noise from given binary images
    def HistogramThresholding(self, img, xsteps=20, ysteps=40, window_width=10):
    
        xstride = img.shape[0] // xsteps
        ystride = img.shape[1] // ysteps
        for xstep in range(xsteps):
            histogram = np.sum(img[xstride*xstep : xstride*(xstep+1), :], axis=0)
            boundary = int(img.shape[1] / 2)
            leftindex = self.GetMaxIndexHistogram(histogram, 0, boundary, window_width=window_width)
            rightindex = self.GetMaxIndexHistogram(histogram, boundary, img.shape[1], window_width=window_width)

            # mask the image
            if histogram[leftindex] >= 3:
                img[xstride*xstep : xstride*(xstep+1), : leftindex-ysteps] = 0
                img[xstride*xstep : xstride*(xstep+1), leftindex+ysteps+1 : boundary] = 0
            else:
                img[xstride*xstep : xstride*(xstep+1), : boundary] = 0

            if histogram[rightindex] >= 3:
                img[xstride*xstep : xstride*(xstep+1), boundary :rightindex-ysteps] = 0
                img[xstride*xstep : xstride*(xstep+1), rightindex+ysteps+1 :] = 0
            else:
                img[xstride*xstep : xstride*(xstep+1), boundary : ] = 0

        left_fit_line, left_line_equation = self.CalculatePolynomial(img, 0, boundary)
        right_fit_line, right_line_equation = self.CalculatePolynomial(img, boundary, img.shape[1])

        # Return binary image after histogram thresholding
        return img, left_fit_line, right_fit_line, left_line_equation, right_line_equation

# Given an image, left_boundary, right_boundary, this function calculates and fits the polynomial on it
def CalculatePolynomial(img, left_boundary, right_boundary):
    side_img = img[:, left_boundary: right_boundary]
    index = np.where(side_img == 1)
    yvals = index[0]
    xvals = index[1] + left_boundary
    fit_equation = np.polyfit(yvals, xvals, 2)
    yvals = np.arange(img.shape[0])
    fit_line = fit_equation[0]*yvals**2 + fit_equation[1]*yvals + fit_equation[2]
    return fit_line

In [0]:
#visualization
try:
    calibration_param = InputCalibrationFile()
except IOError as e:
    print(e)
else:
    mtx = calibration_param['mtx']
    dist = calibration_param['dist']
    
    src = np.array([[490, 482],[810, 482],
                  [1250, 720],[40, 720]], dtype=np.float32)
    dst = np.array([[0, 0], [1280, 0], 
                     [1250, 720],[40, 720]], dtype=np.float32)
    
    BirdViewTransformer = PerspectiveTransform(src, dst)

    for path in glob.glob("./test_images/test*.jpg"):
        fig, axes = plt.subplots(2, 3, figsize=(10, 4))
        distorted_img = mpimg.imread(path)
        undist_img = get_undistortion(distorted_img, mtx, dist)
        bird_view = BirdViewTransformer.Transform(undist_img)            
        s_binary, slope, conversion_img = ColorSlopeThresConversion(bird_view)
        
        a = conversion_img.copy()
        
        final_img, left_lines, right_lines, yvals = Histogram(a, xsteps=20, ysteps=25, window_width=15)
        new_image = np.zeros_like(final_img)
        for yv, ll in zip(yvals, left_lines):
            new_image[yv, ll-10:ll+10] = 1
        for yv, rl in zip(yvals, right_lines):
            new_image[yv, rl-10 : rl+10] = 1
        
        axes[0, 0].imshow(undist_img)
        axes[0, 0].set_title("Undistorted Image")
        axes[0, 1].imshow(bird_view)
        axes[0, 1].set_title("Bird's View")
        axes[0, 2].imshow(s_binary, cmap='gray')
        axes[0, 2].set_title("S Binary")
        axes[1, 0].imshow(slope, cmap='gray')
        axes[1, 0].set_title("Slope Binary")
        axes[1, 1].imshow(conversion_img, cmap='gray')
        axes[1, 1].set_title("Combined Binary")
        axes[1, 2].imshow(final_img, cmap='gray')
        axes[1, 2].set_title("Apply Histogram Filtering")
        
        fig.tight_layout()
        [ax.axis('off') for axe in axes for ax in axe]
        fig.subplots_adjust(left=None, right=None, top=1, bottom=None, wspace=0.1, hspace=0.1)